In [46]:
import numpy as np
import sympy as sp

In [47]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

### Check_convergence
This subroutine is called after each qce_iteration. 
It uses the Gibbs energy to check if the optimization has converged.
The Gibbs energy is used since it depends on the volume and the populations of the clusters.

$$\begin{aligned} G &= - \frac{\ln{Q}}{\beta} + PV \\
                    &= - \frac{\ln{\frac{1}{N!}q^N}}{\beta} + PV \\
                    &= k_{\text{B}}T \bigg( \ln{N!} - N \ln{q} \bigg) +PV \end{aligned}$$

In [48]:
def calc_gibbs(temp, vol, pressure, populations, lnq):
    
    gibbs = 0.0
    for i in range(len(populations)):
        gibbs += kb*temp*(sp.log(sp.factorial(populations[i])) - populations[i]*lnq[i])
        
    gibbs += pressure*vol
    
    return gibbs

#### Tests

In [49]:
temp = 298.15
vol = 1.0e-3
pressure = 1.0e5
populations = np.array([5.2e23, 1.1e23, 3.8e23, 2.3e23])
lnq = np.array([5.3, -5, 38.2, -1.2])

gibbs = calc_gibbs(temp, vol, pressure, populations, lnq)
print(gibbs)

204069.414303223


In [64]:
np.set_printoptions(precision=12)
temp = 512.15
vol = 1.0e-5
pressure = 1.0e7
populations = np.array([5.8e22, 1.1e20, 3.1e25, 2.3e12, 1.0e15])
lnq = np.array([25.3, -51.0, 38.2, -1.2, 111.0])

gibbs = calc_gibbs(temp, vol, pressure, populations, lnq)
print("{:.10f}".format(gibbs))

4284429.7080558800


## Downhill simplex
The downhill simplex is a method developed by Nelder and Mead for geometrical optimization.
It is a numerical method to find the minimum (or maximum) of a multidimensional function.
It is a direct search method. It calculates the value of the function and compares it to other values.
It does not use derivatives. The search method works well but is not guaranteed to converge. (Can get stuck in flat areas) </br></br>
The Nelder-Mead method uses a simplex which is a shape consisting of $n$+1 vertices in $n$ dimensions: </br>

    * 2-D function: simplex is a triangle
    * 3-D function: simplex is a tetrahedron
    * ...

The triangle is used for visualization. </br>
We have a function in 2D space, $f(x,y)$. We want to find a local minimum.
We start with three arbitrary selected points to form our simplex.
The steps for the method are as follows:

1. Sort
2. Reflect
3. Extend
4. Contract
5. Shrink
6. Check convergence

1. $\textbf{Sort}$ </br>
Evalueate the function $f$ at the three points of the simplex.
Sort and label the three points $\textbf{u}$, $\textbf{v}$ and $\textbf{w}$ according to the functions value.
That is: </br>
$$f(u) < f(v) < f(w)$$
According to the function, $\textbf{u}$ gives the smallest value, then $\textbf{v}$ and then $\textbf{w}$. </p>
![Triangle](figures/triangle.png)


2. $\textbf{Reflect}$ </br>
Reflect the worst point $\textbf{w}$ through the centroid of the remaining points $\textbf{u}$ and $\textbf{v}$ to obtain a reflected point $\textbf{r}$. </p>
![Triangle](figures/reflect.png) </p>
Evaluate $f(r)$.
If the cost at the reflected point $f(r)$ is better than $f(v)$ but not better than $f(u)$, then replace $\textbf{w}$ with $\textbf{r}$. With this change, $\textbf{v}$ will become the worst performing point in the simplex. 
Go to step 6 and check convergence.

3. $\textbf{Extend}$ if $f(r)$ < $f(u)$ </br>
If the cost at the reflected point, $f(r)$ is better than $f(u)$, then extend the reflected point $\textbf{r}$ even further to $\textbf{e}$ Evaluate $f(e)$. </p>
![Extend](figures/extend.png) </br>
If the cost at the extended point $f(e)$ is even better than the cost at the reflected point, $f(r)$, replace the worst point $\textbf{w}$ with the extended point $\textbf{e}$. Go to step 6 and check convergence. </p>
If the cost at the extended point $f(e)$ is not better than the cost at the reflected point, $f(r)$, replace the worst point $\textbf{w}$ with the reflected point $\textbf{r}$. Go to step 6 and check convergence.


4. $\textbf{Contract}$ If the checks in step 2 or step 3 are not satisfied </br>
The reflected point $\textbf{r}$ is not better than $\textbf{v}$ or $\textbf{u}$.
Contract the worst point $\textbf{w}$ to the points $\textbf{c}_1$ and $\textbf{c}_0$ along the line of reflection.
    * $\textbf{c}_1$ is the inside contracted point and is 1/4 of the way from $\textbf{w}$ to $\textbf{r}$.
    * $\textbf{c}_0$ is the outside contracted point and is 3/4 of the way from $\textbf{w}$ to $\textbf{r}$. </p>
![Contract](figures/contract.png) </br> 
Evaluate the function at both locations. If either of these points performs better than $f(v)$, then replace $\textbf{w}$ with the better performing point. Go to step 6 and check convergence. 

5. $\textbf{Shrink}$ if neither of the contracted points performs better than $\textbf{v}$ </br>
We shrink the simplex towards the best point $\textbf{u}$.
* Replace $\textbf{w}$ with $\textbf{w}'$ which is 1/2 of the way from $\textbf{w}$ to $\textbf{u}$.
* Replace $\textbf{v}$ with $\textbf{v}'$ which is 1/2 of the way from $\textbf{v}$ to $\textbf{u}$. </p>
![Shrink](figures/shrink.png) </br>

6. $\textbf{Check convergence}$ </br>
Calculate the difference between the best and worst point in the simplex. If the difference is less than a certain tolerance, the algorithm has converged. </br>